In [1]:
# Run this first in Colab if packages missing.
# (Uncomment the pip line if running in a clean environment)
# !pip install -q statsmodels scikit-learn matplotlib seaborn

import os
print("Ready. If you need to install, uncomment the pip line above.")


Ready. If you need to install, uncomment the pip line above.


In [2]:
import os, json, math
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
from scipy.special import expit, logit

PROJECT = "/content/dml_tml_project"
PLOTS = os.path.join(PROJECT, "plots")
os.makedirs(PLOTS, exist_ok=True)
print("Project folder:", PROJECT)
print("Plots folder:", PLOTS)


Project folder: /content/dml_tml_project
Plots folder: /content/dml_tml_project/plots


In [3]:
np.random.seed(42)
n = 12000  # moderate size under 5MB CSV
# covariates
W = pd.DataFrame({
    "x1": np.random.normal(0,1,n),
    "x2": np.random.normal(0,1,n),
    "x3": np.random.randint(0,4,n).astype(float),
    "x4": np.random.choice([0,1], size=n, p=[0.6,0.4]),   # binary cov
})
# true propensity (depends on W)
logit_g = -0.2 + 0.6*W["x1"] - 0.3*W["x3"] + 0.4*W["x4"]
prob_t = 1/(1+np.exp(-logit_g))
T = np.random.binomial(1, prob_t, size=n)

# true treatment effect heterogeneous: depends on x1 & x4
true_tau = 0.8*(W["x1"]>0).astype(float) + 0.5*W["x4"]

# baseline outcome
mu0 = 0.5*W["x1"] - 0.3*W["x2"] + 0.1*W["x3"] - 0.4*W["x4"] + np.random.normal(0,0.6,n)
# potential outcomes
Y0 = (logit(mu0) if False else mu0)  # keep continuous latent; we'll make binary outcome
Y1_cont = mu0 + true_tau
# convert to probability via logistic and draw binary outcome
p0 = expit(Y0)
p1 = expit(Y1_cont)
Y = np.array([np.random.binomial(1, p1[i]) if T[i]==1 else np.random.binomial(1, p0[i]) for i in range(n)])

df = pd.concat([W, pd.Series(T, name="treatment"), pd.Series(Y, name="outcome")], axis=1)
csv_path = os.path.join(PROJECT, "synthetic_causal_dataset.csv")
df.to_csv(csv_path, index=False)
print("Saved synthetic dataset:", csv_path)
print("Default sizes: n =", len(df), "treatment rate:", df['treatment'].mean())


Saved synthetic dataset: /content/dml_tml_project/synthetic_causal_dataset.csv
Default sizes: n = 12000 treatment rate: 0.39166666666666666


In [4]:
# Basic EDA: covariate means by treatment
summary = df.groupby("treatment")[["x1","x2","x3","x4"]].mean().T
print("Mean covariates by treatment:\n", summary)

# Standardized mean differences (SMD)
def smd(df, col, treat_col="treatment"):
    m1 = df.loc[df[treat_col]==1, col].mean()
    m0 = df.loc[df[treat_col]==0, col].mean()
    s = np.sqrt((df[col].var()))
    return (m1 - m0)/s

smds = {c: smd(df, c) for c in ["x1","x2","x3","x4"]}
print("SMDs (pre-adjustment):", smds)

# Plot distributions for x1,x2 by treatment
for col in ["x1","x2"]:
    plt.figure(figsize=(6,3))
    sns.kdeplot(df.loc[df.treatment==0, col], label="control", bw_adjust=1.2)
    sns.kdeplot(df.loc[df.treatment==1, col], label="treated", bw_adjust=1.2)
    plt.title(f"Distribution of {col} by treatment")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS, f"dist_{col}_by_treatment.png"))
    plt.close()
print("Saved EDA plots to", PLOTS)


Mean covariates by treatment:
 treatment         0         1
x1        -0.215736  0.319662
x2         0.001832  0.022126
x3         1.644384  1.300638
x4         0.360685  0.455106
SMDs (pre-adjustment): {'x1': np.float64(0.5345057055574404), 'x2': np.float64(0.020303977031750072), 'x3': np.float64(-0.3091487783199045), 'x4': np.float64(0.1929187913997293)}
Saved EDA plots to /content/dml_tml_project/plots


In [5]:
X = df[["x1","x2","x3","x4"]]
y_t = df["treatment"]
ps_model = LogisticRegression(max_iter=2000)
ps_model.fit(X, y_t)
propensity = ps_model.predict_proba(X)[:,1]
df["propensity"] = propensity

plt.figure(figsize=(6,4))
sns.histplot(df[df.treatment==1]['propensity'], label="treated", kde=True, stat="density", color="C1")
sns.histplot(df[df.treatment==0]['propensity'], label="control", kde=True, stat="density", color="C0")
plt.legend(); plt.title("Propensity score distribution")
plt.tight_layout()
plt.savefig(os.path.join(PLOTS,"propensity_distribution.png")); plt.close()
print("Saved propensity distribution.")


Saved propensity distribution.


In [6]:
# stabilized weights
p = df["propensity"].values
t = df["treatment"].values
# stabilized numerator: marginal probability of treatment
p_t = t.mean()
p_c = 1 - p_t
stabilized_w = np.where(t==1, p_t / p, p_c / (1-p))
df["ipw_stabilized"] = stabilized_w

# SMD after weighting (weighted means)
def weighted_mean(df, col, weight_col, group_val):
    w = df.loc[df.treatment==group_val, weight_col]
    x = df.loc[df.treatment==group_val, col]
    return (w * x).sum() / w.sum()

smd_post = {}
for c in ["x1","x2","x3","x4"]:
    m1 = weighted_mean(df, c, "ipw_stabilized", 1)
    m0 = weighted_mean(df, c, "ipw_stabilized", 0)
    pooled_sd = np.sqrt(((df[c].var())))
    smd_post[c] = (m1 - m0)/pooled_sd
print("SMD post-weighting:", smd_post)

# plot SMD before/after
cols = ["x1","x2","x3","x4"]
before = [smds[c] for c in cols]
after = [smd_post[c] for c in cols]
xpos = np.arange(len(cols))
plt.figure(figsize=(6,3))
plt.bar(xpos-0.15, before, width=0.3, label="before")
plt.bar(xpos+0.15, after, width=0.3, label="after")
plt.axhline(0.1, color='red', linestyle='--', linewidth=0.7)
plt.xticks(xpos, cols)
plt.ylabel("Standardized mean diff")
plt.legend()
plt.title("Covariate balance before/after IPW")
plt.tight_layout()
plt.savefig(os.path.join(PLOTS,"smd_before_after.png"))
plt.close()
print("Saved covariate balance plot.")


SMD post-weighting: {'x1': np.float64(0.006101721855257966), 'x2': np.float64(0.0056609049736738165), 'x3': np.float64(-0.00557020019440208), 'x4': np.float64(0.005810109628728016)}
Saved covariate balance plot.


In [7]:
from sklearn.base import clone
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

# learners
m_model = GradientBoostingRegressor(n_estimators=200, max_depth=3, random_state=1)  # outcome model
g_model = GradientBoostingClassifier(n_estimators=200, max_depth=3, random_state=2)  # propensity

n = len(df)
theta_vals = []
psi_list = []

# arrays to store predictions
m_hat = np.zeros(n)
g_hat = np.zeros(n)

X_vals = df[["x1","x2","x3","x4"]].values
Y_vals = df["outcome"].values
T_vals = df["treatment"].values

for train_idx, test_idx in kf.split(X_vals):
    # fit nuisance models on train
    m = clone(m_model)
    g = clone(g_model)
    m.fit(X_vals[train_idx], Y_vals[train_idx])
    g.fit(X_vals[train_idx], T_vals[train_idx])
    # predict on test
    m_hat[test_idx] = m.predict(X_vals[test_idx])
    g_hat[test_idx] = g.predict_proba(X_vals[test_idx])[:,1]

# residuals
res_y = Y_vals - m_hat
res_t = T_vals - g_hat

# final theta via linear regression of res_y on res_t (simple OLS)
theta = np.sum(res_t * res_y) / np.sum(res_t * res_t)
# compute std err (influence function)
psi = res_t * (res_y - theta * res_t)
se = np.std(psi) / np.sqrt(n)
ci_low = theta - 1.96*se
ci_high = theta + 1.96*se

print("DML ATE estimate:", theta, "SE:", se, "95% CI:", (ci_low,ci_high))
df_results = {"dml_ate": float(theta), "dml_se": float(se), "dml_ci": [float(ci_low), float(ci_high)]}


DML ATE estimate: 0.11833601067442741 SE: 0.0020232097132245274 95% CI: (np.float64(0.11437051963650734), np.float64(0.12230150171234748))


In [8]:
# We'll implement TMLE for ATE (binary outcome) using logistic fluctuation

# 1) fit initial outcome regression Q(A,W) predicting E[Y|A,W]
X_aw = np.hstack([X_vals, T_vals.reshape(-1,1)])  # features + treatment
Q_model = GradientBoostingRegressor(n_estimators=200, max_depth=3, random_state=3)
Q_model.fit(X_aw, Y_vals)  # fits for outcome on A and W

# predict Q0 and Q1 (for all obs)
X0 = np.hstack([X_vals, np.zeros((n,1))])
X1 = np.hstack([X_vals, np.ones((n,1))])
Q0 = Q_model.predict(X0)
Q1 = Q_model.predict(X1)
# truncate to (eps,1-eps) after passing through logit if necessary; we'll use expit(logit) later
# estimate initial ATE (plug-in)
psi_initial = (Q1 - Q0).mean()

# 2) fit propensity g (already have g_hat from DML - but re-use full-sample fit)
g_full = clone(g_model)
g_full.fit(X_vals, T_vals)
g_full_p = g_full.predict_proba(X_vals)[:,1]

# 3) clever covariate H = A/g + (1-A)/ (1-g) ?? For ATE use H = A/g - (1-A)/(1-g)
H = (T_vals / g_full_p) - ((1-T_vals) / (1 - g_full_p))

# 4) logistic fluctuation: transform Q (continuous predictions) to logit-scale probabilities
# ensure values in (eps,1-eps)
eps = 1e-6
q0_p = np.clip(Q_model.predict(X_aw) , 1e-3, 1-1e-3)  # but Q_model returns continuous; instead use expit(Q)
# better: treat Q0/Q1 as logits? We'll map predicted continuous to probability via expit
Q0_p = np.clip(expit(Q0), 1e-6, 1-1e-6)
Q1_p = np.clip(expit(Q1), 1e-6, 1-1e-6)
Q_p = np.clip(expit(Q_model.predict(X_aw)), 1e-6, 1-1e-6)

# Fit epsilon via weighted logistic regression with offset = logit(Q_p) and covariate H
# Using statsmodels: endog = Y, exog = H (constant included), offset = logit(Q_p)
logit_Q = np.log(Q_p/(1-Q_p))
# add small constant to offset to avoid inf
exog = sm.add_constant(H)  # include intercept and H
glm_binom = sm.GLM(Y_vals, exog, family=sm.families.Binomial(), offset=logit_Q)
res_glm = glm_binom.fit()
# epsilon = coefficient on H (exog column 1)
eps_hat = res_glm.params[1]
print("Estimated fluctuation epsilon:", eps_hat)

# update Q* = expit(logit(Q_p) + epsilon * H)
Q_star = expit(logit_Q + eps_hat * H)

# Now compute targeted estimates of Q1* and Q0* by adjusting individual-level predictions:
# For each observation, compute updated Q1_star_i and Q0_star_i as:
# Q1_star_i = expit( logit(Q1_p_i) + eps_hat * (1/g_i) )
# Q0_star_i = expit( logit(Q0_p_i) + eps_hat * (-1/(1-g_i)) )
Q1_star = expit(np.log(Q1_p/(1-Q1_p)) + eps_hat * (1.0 / g_full_p))
Q0_star = expit(np.log(Q0_p/(1-Q0_p)) + eps_hat * (-1.0 / (1 - g_full_p)))

tmle_psi = np.mean(Q1_star - Q0_star)
# approximate std error using influence function
IF = ((T_vals / g_full_p) * (Y_vals - Q_p)) + (Q1_star - Q0_star - tmle_psi)
tmle_se = np.std(IF) / np.sqrt(n)
tmle_ci = (tmle_psi - 1.96*tmle_se, tmle_psi + 1.96*tmle_se)

print("TMLE ATE:", tmle_psi, "SE:", tmle_se, "95% CI:", tmle_ci)
df_results.update({"tmle_ate": float(tmle_psi), "tmle_se": float(tmle_se), "tmle_ci": [float(tmle_ci[0]), float(tmle_ci[1])]})


Estimated fluctuation epsilon: 0.08131834331419424
TMLE ATE: 0.11559623240567293 SE: 0.007277577888057607 95% CI: (np.float64(0.10133217974508002), np.float64(0.12986028506626585))


In [9]:
sub_mask = df["x1"] > 0
print("Subgroup size:", sub_mask.sum(), "total:", len(df))
# DML on subgroup (simple: re-run DML steps restricted to subgroup)
X_sub = df.loc[sub_mask, ["x1","x2","x3","x4"]].values
Y_sub = df.loc[sub_mask, "outcome"].values
T_sub = df.loc[sub_mask, "treatment"].values
# fit nuisance on subgroup full-sample (no cross-fitting here for speed) -- acceptable for report but note in writeup
m = GradientBoostingRegressor(n_estimators=200, max_depth=3)
g = GradientBoostingClassifier(n_estimators=200, max_depth=3)
m.fit(np.hstack([X_sub, T_sub.reshape(-1,1)]), Y_sub)  # outcome on A,W
g.fit(X_sub, T_sub)
m_hat_sub = m.predict(np.hstack([X_sub, T_sub.reshape(-1,1)]))
g_hat_sub = g.predict_proba(X_sub)[:,1]
res_y_sub = Y_sub - m_hat_sub
res_t_sub = T_sub - g_hat_sub
theta_sub = np.sum(res_t_sub * res_y_sub)/np.sum(res_t_sub * res_t_sub)
se_sub = np.std(res_t_sub * (res_y_sub - theta_sub*res_t_sub)) / np.sqrt(sub_mask.sum())
ci_sub = (theta_sub - 1.96*se_sub, theta_sub + 1.96*se_sub)

# TMLE on subgroup (similar to above but restricted)
g_sub = g
g_sub_p = g_sub.predict_proba(X_sub)[:,1]
# initial Q on subgroup
Qm = GradientBoostingRegressor(n_estimators=200, max_depth=3)
Qm.fit(np.hstack([X_sub, T_sub.reshape(-1,1)]), Y_sub)
Q0_sub = expit(Qm.predict(np.hstack([X_sub, np.zeros((len(X_sub),1))])))
Q1_sub = expit(Qm.predict(np.hstack([X_sub, np.ones((len(X_sub),1))])))
# estimate eps via glm
H_sub = (T_sub / g_sub_p) - ((1-T_sub) / (1 - g_sub_p))
Qp_sub = np.clip(expit(Qm.predict(np.hstack([X_sub, T_sub.reshape(-1,1)]))),1e-6,1-1e-6)
logit_Qp_sub = np.log(Qp_sub/(1-Qp_sub))
exog_sub = sm.add_constant(H_sub)
glm_binom_sub = sm.GLM(Y_sub, exog_sub, family=sm.families.Binomial(), offset=logit_Qp_sub)
res_glm_sub = glm_binom_sub.fit()
eps_sub = res_glm_sub.params[1]
Q1_star_sub = expit(np.log(Q1_sub/(1-Q1_sub)) + eps_sub * (1.0 / g_sub_p))
Q0_star_sub = expit(np.log(Q0_sub/(1-Q0_sub)) + eps_sub * (-1.0 / (1 - g_sub_p)))
tmle_sub = np.mean(Q1_star_sub - Q0_star_sub)
IF_sub = ((T_sub / g_sub_p) * (Y_sub - Qp_sub)) + (Q1_star_sub - Q0_star_sub - tmle_sub)
tmle_sub_se = np.std(IF_sub)/np.sqrt(len(X_sub))
tmle_sub_ci = (tmle_sub - 1.96*tmle_sub_se, tmle_sub + 1.96*tmle_sub_se)

print("Subgroup DML ATE:", theta_sub, "SE:", se_sub, "CI:", ci_sub)
print("Subgroup TMLE ATE:", tmle_sub, "SE:", tmle_sub_se, "CI:", tmle_sub_ci)

df_results.update({
    "sub_dml_ate": float(theta_sub), "sub_dml_se": float(se_sub), "sub_dml_ci": [float(ci_sub[0]), float(ci_sub[1])],
    "sub_tmle_ate": float(tmle_sub), "sub_tmle_se": float(tmle_sub_se), "sub_tmle_ci": [float(tmle_sub_ci[0]), float(tmle_sub_ci[1])]
})


Subgroup size: 5964 total: 12000
Subgroup DML ATE: 0.00887137989735717 SE: 0.0025210283511936334 CI: (np.float64(0.003930164329017648), np.float64(0.013812595465696692))
Subgroup TMLE ATE: 0.21055520200649794 SE: 0.007615623961895639 CI: (np.float64(0.1956285790411825), np.float64(0.22548182497181338))


In [10]:
# Save results JSON
results_path = os.path.join(PROJECT, "results_summary.json")
with open(results_path, "w") as f:
    json.dump(df_results, f, indent=2)
print("Saved results to", results_path)

# Example plot: compare ATE estimates bar
plt.figure(figsize=(6,3))
methods = ["DML","TMLE","Sub-DML","Sub-TMLE"]
vals = [df_results["dml_ate"], df_results["tmle_ate"], df_results["sub_dml_ate"], df_results["sub_tmle_ate"]]
errs = [df_results["dml_se"]*1.96, df_results["tmle_se"]*1.96, df_results["sub_dml_se"]*1.96, df_results["sub_tmle_se"]*1.96]
plt.bar(methods, vals, yerr=errs, capsize=6)
plt.ylabel("ATE estimate")
plt.title("ATE estimates (95% CI)")
plt.tight_layout()
plt.savefig(os.path.join(PLOTS,"ate_comparison.png"))
plt.close()
print("Saved ATE comparison plot to", PLOTS)


Saved results to /content/dml_tml_project/results_summary.json
Saved ATE comparison plot to /content/dml_tml_project/plots
